In [2]:
from openai import AzureOpenAI

api_key = "a1ab70e0f13c4db1b2236ae9ea986511"
api_version = "2024-05-01-preview"

vectorClient = AzureOpenAI(
    api_key=api_key,
    azure_endpoint="https://james-ai.openai.azure.com",
    api_version=api_version
)

In [3]:
# Create a vector store called "Financial Statements"
vector_store = vectorClient.beta.vector_stores.create(name="Stories")

vector_store

VectorStore(id='vs_COnJo0nH42LDNQ86Ir3F6KT8', created_at=1716653379, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1716653379, metadata={}, name='Stories', object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None)

In [4]:
# Ready the files for upload to OpenAI
file_paths = ["files/Alice_1.txt"]
file_streams = [open(path, "rb") for path in file_paths]

In [5]:
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = vectorClient.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

In [6]:
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [7]:
assistantClient = AzureOpenAI(
    api_key=api_key,
    azure_endpoint="https://james-ai.openai.azure.com/openai/assistants?api-version=2024-05-01-preview",
    api_version=api_version
)

assistant = assistantClient.beta.assistants.create(
  name="AIWL Assistant",
  instructions="You are an expert in the story of Alice in Wonderland. Use your knowledge base to answer questions about the story.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)


In [8]:
threadsClient = AzureOpenAI(
    api_key=api_key,
    azure_endpoint="https://james-ai.openai.azure.com/openai/threads?api-version=2024-05-01-preview",
    api_version=api_version
)

# Create a thread
thread = threadsClient.beta.threads.create()

thread

Thread(id='thread_q50yuvYzyMpa4VtrDLxXaHQ1', created_at=1716653390, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [9]:
messageClient = AzureOpenAI(
    api_key=api_key,
    azure_endpoint=f"https://james-ai.openai.azure.com/openai/threads/{thread.id}/messages?api-version=2024-05-01-preview",
    api_version=api_version
)

# Add a user question to the thread
message = messageClient.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Who are the characters in the story?"
)

message

Message(id='msg_JqHRRuhjB1N418P0zF451g7b', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Who are the characters in the story?'), type='text')], created_at=1716653392, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_q50yuvYzyMpa4VtrDLxXaHQ1')

In [10]:
import time

runClient = AzureOpenAI(
    api_key=api_key,
    azure_endpoint=f"https://james-ai.openai.azure.com/openai/threads/{thread.id}/runs?api-version=2024-05-01-preview",
    api_version=api_version
)

# Run the thread
run = runClient.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = runClient.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
  print(run.status)

None


In [13]:
messages = messageClient.beta.threads.messages.list(
  thread_id=thread.id
)

messages.data[0]

Message(id='msg_jVTSxKf5jZgDc0D0nmgK1ra5', assistant_id='asst_7lfrKTxU6uNvMzF4JeMo89cr', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Here is a list of characters in "Alice\'s Adventures in Wonderland":\n\n1. **Alice** - The protagonist of the story who falls down a rabbit hole into Wonderland.\n2. **The White Rabbit** - The rabbit that Alice follows down the rabbit hole, recognizable by its waistcoat and pocket watch.\n3. **The Queen of Hearts** - A tyrannical ruler who is quick to order executions.\n4. **The King of Hearts** - The Queen\'s husband, who is much less aggressive than the Queen.\n5. **The Duchess** - A character who has a sharp chin and enjoys moralizing.\n6. **The Cheshire Cat** - Known for its distinctive grin and ability to disappear and reappear at will.\n7. **The Mad Hatter** - A mad character who hosts a perpetual tea party.\n8. **The March Hare** - Companion to the Mad Hatter at their never-ending tea party.\n9. **T